# PZQ-R field variants


We score and investigate variants from natural populations in order to have a comprehensive view of those that could impact praziquantel efficacy. We used data from different locations in New and Old World:
* Brazil (original from the associated publication)
* Senegal
* Niger
* Tanzania
* Uganda (data generated by Sanger https://doi.org/10.3389/fgene.2019.00826)
* Oman


## Environment and data

### Environment

Creating a conda environment improves reproducibility by installing specific versions of the programs used.

In [ ]:
conda env create -f .env/env.yml

The cell below must be run each time a new Jupyter session is run or when the kernel is rebooted.

In [2]:
# Activate the environment
source $(sed "s,/bin/conda,," <<<$CONDA_EXE)/etc/profile.d/conda.sh
conda activate PZQ-R_field

# Remove potential variable interferences
export PERL5LIB=""
export PYTHONNOUSERSITE=1

(PZQ-R_field) (PZQ-R_field) (PZQ-R_field) (PZQ-R_field) (PZQ-R_field) 

: 1

The cell below must be run only once at the time of the environment creation. The R script install the R dependencies.

In [ ]:
# Installing needed R packages
Rscript ".env/R package dependencies.R"

#### Phred/Phrap/Consed and PolyPhred

[Phred/Phrap/Consed](http://www.phrap.org/phredphrapconsed.html) and [PolyPhred](https://droog.gs.washington.edu/polyphred/) are software used to analyze Sanger sequencing traces by performing alignment and genotyping, respectively. They are freely available for academic used. However, this requires to contact the authors in order to get a copy ([here](http://www.phrap.org/consed/consed.html#howToGet) for Phred/Phrap/Consed, [here](https://droog.gs.washington.edu/polyphred/poly_get.html) for PolyPhred).

Once the copies obtained, they must be place in a download folder at the same level as this notebook. Then programs can be compiled.

In [ ]:
# Folder in which the software archives must be located
ddir="download"
[[ ! -d "$ddir" ]] && mkdir -p "$ddir"

##### Phred

Below is the procedure to compile phred and daev. This requires to have access to a terminal.

In [ ]:
myphred="phred-dist-020425.c-acd.tar.Z"

## Decompress archive
[[ ! -d "$ddir/phred" ]] && mkdir -p "$ddir/phred"
tar -C "$ddir/phred" -xzvf "$ddir/$myphred"

In [ ]:
# Copy the executables to the conda environment
cp -a "$ddir/phred/phred" "$ddir/phred/daev" "$CONDA_PREFIX/bin/"

# Copy the needed file
cp -a "$ddir/phred/phredpar.dat" "$CONDA_PREFIX/etc/phredpar.dat"

In [ ]:
# Add the machine and chemistry used by the sequencing facility to the phredpar file
## Machine model and related information are available in the header of scf files
sed -i '/^begin/,/^# Chromatograms/ s/^#$/"KB_3730_POP7_BDTv3.mob"        terminator      big-dye                 ABI_3700\n#/' "$CONDA_PREFIX/etc/phredpar.dat"

##### Phd2fasta

Below is the procedure to compile phd2fasta. This requires to have access to a terminal.

In [ ]:
myphd="phd2fasta-acd-dist.130911.tar.gz"

## Decompress archive
[[ ! -d "$ddir/phd" ]] && mkdir -p "$ddir/phd"
tar -C "$ddir/phd" -xzvf "$ddir/$myphd"

In [ ]:
# Copy the executable to the conda environment
cp -a "$ddir/phd/phd2fasta" "$CONDA_PREFIX/bin/"

##### Phrap

Below is the procedure to compile phrap. This requires to have access to a terminal.

In [ ]:
myphrap="distrib.tar.Z"

## Decompress archive
[[ ! -d "$ddir/phrap.d" ]] && mkdir -p "$ddir/phrap.d"
tar -C "$ddir/phrap.d" -xzvf "$ddir/$myphrap"

In [ ]:
# Clean and copy the folder to the conda environment
rm "$ddir/phrap.d/"*.{c,h,o}
cp -aR "$ddir/phrap.d/" "$CONDA_PREFIX/bin/"

In [ ]:
ln -s "$CONDA_PREFIX/bin/phrap.d/"{cross_match,cross_match.manyreads,loco,phrap,phrap.longreads,phrap.manyreads,phrapview,swat} "$CONDA_PREFIX/bin/"

##### Consed

Below is the procedure to compile consed. This requires to have access to a terminal.

In [ ]:
mycs="consed_linux.tar.gz"

## Decompress archive
[[ ! -d "$ddir/consed" ]] && mkdir -p "$ddir/consed"
tar -C "$ddir/consed" -xzvf "$ddir/$mycs"

In [ ]:
for i in "$ddir/consed/"consed*
do
    ./"$i" -v
    [[ $? -eq 0 ]] && prog=$(basename "$i") && break
done

In [ ]:
# Correct small bug in install script (when whitespace is present in current path)
sed -i 's|cp $szDownloadDirectory|cp \\"$szDownloadDirectory\\"|g' "$ddir/consed/installConsed.perl"
sed -i 's|cp $szNewDir|cp \\"$szNewDir\\"|g' "$ddir/consed/installConsed.perl"

In [ ]:
ln -s "$CONDA_PREFIX/bin/phrap.d/"{cross_match,cross_match.manyreads} "$CONDA_PREFIX/bin/consed.d/bin"

In [29]:
ln -s "$CONDA_PREFIX/bin/phred" "$CONDA_PREFIX/bin/consed.d/bin"

(PZQ-R_field) 

: 1

In [31]:
ln -s "$CONDA_PREFIX/etc/phredpar.dat" "$CONDA_PREFIX/bin/consed.d/lib"

(PZQ-R_field) 

: 1

##### PolyPhred

Below is the procedure to compile polyphred. This requires to have access to a terminal.

In [ ]:
mypp="polyphred-6.18-binary-x86_64-unknown-linux-gnu.tar.gz"

## Decompress archive
tar -C "$CONDA_PREFIX/bin/" -xzvf "$ddir/$mypp"

# Link executable
ln -s "$CONDA_PREFIX/bin/polyphred-6.18-binary-x86_64-unknown-linux-gnu/bin/"* "$CONDA_PREFIX/bin/"

In [ ]:
# Correct perl location
sed -i '1s|^#!.*|#!/usr/bin/env perl|' "$CONDA_PREFIX/bin/polyphred-6.18-binary-x86_64-unknown-linux-gnu/bin/phredPhrap.pl"

##### Environment variable

In order to have phred and consed working as expected, two environment variables need to be exported each time a new Jupyter session is run or when the kernel is rebooted.

In [35]:
export PHRED_PARAMETER_FILE="$CONDA_PREFIX/etc/phredpar.dat"
export CONSED_HOME="$CONDA_PREFIX/bin/consed.d"

(PZQ-R_field) (PZQ-R_field) 

: 1

### Sequencing data

This step downloads the fastq files of the different samples from the SRA repository.

In [7]:
# Data directory
ldir="data/libraries"
[[ ! -d "$ldir" ]] && mkdir -p "$ldir"

# Data directory for S. mansoni
ldir_sm="$ldir/1-Sm"
[[ ! -d "$ldir_sm" ]] && mkdir -p "$ldir_sm"

# Data directory for S. haematobium
ldir_sh="$ldir/2-Sh"
[[ ! -d "$ldir_sh" ]] && mkdir -p "$ldir_sh"

(PZQ-R_field) (PZQ-R_field) (PZQ-R_field) (PZQ-R_field) (PZQ-R_field) (PZQ-R_field) (PZQ-R_field) (PZQ-R_field) (PZQ-R_field) (PZQ-R_field) (PZQ-R_field) 

: 1

In [ ]:
# Bioproject
bioproject=ERP114942

# Download related information to data project
wget -q -O runinfo "http://trace.ncbi.nlm.nih.gov/Traces/sra/sra.cgi?save=efetch&rettype=runinfo&db=sra&term=${bioproject}"

# Field of interest (library name and weblink)
fdn=$(head -n 1 runinfo | tr "," "\n" | grep -w -n "SampleName" | cut -d ":" -f 1)
fdr=$(head -n 1 runinfo | tr "," "\n" | grep -w -n "Run" | cut -d ":" -f 1)
flk=$(head -n 1 runinfo | tr "," "\n" | grep -w -n "download_path" | cut -d ":" -f 1)

# Keep S. mansoni samples only
fds=$(head -1 runinfo | tr "," "\n" | grep -w -n "ScientificName" | cut -d ":" -f 1)
awk -v fds=$fds -F "," '$fds == "Schistosoma mansoni"' runinfo > runinfo_sm

# Download fastq files
while read line
do
    # Filename, run and download link
    fln=$(cut -d "," -f $fdn <<<$line)
    run=$(cut -d "," -f $fdr <<<$line)
    lnk=$(cut -d "," -f $flk <<<$line)
    
    # Download
    echo "$fln"
    [[ ! -d "$ldir_sm/$fln/" ]] && mkdir -p "$ldir_sm/$fln/"
    retry=0
    
    while [[ $retry -lt 2 ]]
    do
        # Download sra file
        wget -q -c -O "$ldir_sm/$fln/$run" "$lnk"
        # Check integrity
        vdb-validate -q "$ldir_sm/$fln/$run" &> /dev/null
        [[ $? -ne 0 ]] && ((retry++)) || break
    done
    
    # If max download attempt reached, issue message and move to the next
    [[ $retry -eq 2 ]] && echo "$run: dowloading problem" >> "$ldir_sm/download_issue" && contine
    
    # Convert sra into fastq
    fastq-dump -O "$ldir_sm/$fln/" --split-files "$ldir_sm/$fln/$run"
    rm "$ldir_sm/$fln/$run"
    
    # Rename file with more meaningful name
    mv "$ldir_sm/$fln/${run}_1.fastq" "$ldir_sm/$fln/${fln}_R1.fastq"
    mv "$ldir_sm/$fln/${run}_2.fastq" "$ldir_sm/$fln/${fln}_R2.fastq"
        
done < runinfo_sm

# Compress files
pigz "$ldir_sm/"*/*

rm runinfo*

Download our exome data

In [ ]:
# Bioproject
bioproject=PRJNA560069

# Download related information to data project
wget -q -O runinfo "http://trace.ncbi.nlm.nih.gov/Traces/sra/sra.cgi?save=efetch&rettype=runinfo&db=sra&term=${bioproject}"

# Field of interest (library name and weblink)
fdn=$(head -n 1 runinfo | tr "," "\n" | grep -w -n "SampleName" | cut -d ":" -f 1)
fdr=$(head -n 1 runinfo | tr "," "\n" | grep -w -n "Run" | cut -d ":" -f 1)
flk=$(head -n 1 runinfo | tr "," "\n" | grep -w -n "download_path" | cut -d ":" -f 1)

# Download fastq files
while read line
do
    # Filename, run and download link
    fln=$(cut -d "," -f $fdn <<<$line)
    run=$(cut -d "," -f $fdr <<<$line)
    lnk=$(cut -d "," -f $flk <<<$line)
    
    # Download
    echo "$fln"
    [[ ! -d "$ldir/$fln/" ]] && mkdir -p "$ldir/$fln/"
    retry=0
    
    while [[ $retry -lt 2 ]]
    do
        # Download sra file
        wget -q -c -O "$ldir/$fln/$run" "$lnk"
        # Check integrity
        vdb-validate -q "$ldir/$fln/$run" &> /dev/null
        [[ $? -ne 0 ]] && ((retry++)) || break
    done
    
    # If max download attempt reached, issue message and move to the next
    [[ $retry -eq 2 ]] && echo "$run: dowloading problem" >> "$ldir/download_issue" && contine
    
    # Convert sra into fastq
    fastq-dump -O "$ldir/$fln/" --split-files "$ldir/$fln/$run"
    rm "$ldir/$fln/$run"
    
    # Rename file with more meaningful name
    mv "$ldir/$fln/${run}_1.fastq" "$ldir/$fln/${fln}_R1.fastq"
    mv "$ldir/$fln/${run}_2.fastq" "$ldir/$fln/${fln}_R2.fastq"
        
done < <(tail -n +2 runinfo | sed "/^$/d")

# Compress files
pigz "$ldir/"*/*

rm runinfo*

*S. haematobium* data

In [5]:
# Bioproject
bioproject=PRJNA443709

# Download related information to data project
wget -q -O runinfo "http://trace.ncbi.nlm.nih.gov/Traces/sra/sra.cgi?save=efetch&rettype=runinfo&db=sra&term=${bioproject}"

# Field of interest (library name and weblink)
fdn=$(head -n 1 runinfo | tr "," "\n" | grep -w -n "SampleName" | cut -d ":" -f 1)
fdr=$(head -n 1 runinfo | tr "," "\n" | grep -w -n "Run" | cut -d ":" -f 1)
flk=$(head -n 1 runinfo | tr "," "\n" | grep -w -n "download_path" | cut -d ":" -f 1)

# Download fastq files
while read line
do
    # Filename, run and download link
    fln=$(cut -d "," -f $fdn <<<$line)
    run=$(cut -d "," -f $fdr <<<$line)
    lnk=$(cut -d "," -f $flk <<<$line)
    
    # Download
    echo "$fln"
    [[ ! -d "$ldir_sh/$fln/" ]] && mkdir -p "$ldir_sh/$fln/"
    retry=0
    
    while [[ $retry -lt 2 ]]
    do
        # Download sra file
        wget -q -c -O "$ldir_sh/$fln/$run" "$lnk"
        # Check integrity
        vdb-validate -q "$ldir_sh/$fln/$run" &> /dev/null
        [[ $? -ne 0 ]] && ((retry++)) || break
    done
    
    # If max download attempt reached, issue message and move to the next
    [[ $retry -eq 2 ]] && echo "$run: dowloading problem" >> "$ldir_sh/download_issue" && contine
    
    # Convert sra into fastq
    fastq-dump -O "$ldir_sh/$fln/" --split-files "$ldir_sh/$fln/$run"
    rm "$ldir_sh/$fln/$run"
    
    # Rename file with more meaningful name
    mv "$ldir_sh/$fln/${run}_1.fastq" "$ldir_sh/$fln/${fln}_R1.fastq"
    mv "$ldir_sh/$fln/${run}_2.fastq" "$ldir_sh/$fln/${fln}_R2.fastq"
        
done < <(tail -n +2 runinfo | sed "/^$/d")

# Compress files
pigz "$ldir_sh/"*/*

rm runinfo

(PZQ-R_field) (PZQ-R_field) (PZQ-R_field) (PZQ-R_field) (PZQ-R_field) 

: 1

### Genome data

The genome data is downloaded from the [WormBase ParaSite](https://parasite.wormbase.org). We use the data from the version 14 (WBPS14) for *S. mansoni* and version 15 (WBPS14) for *S. haematobium*. The data is then indexed for the different tools used.

In [3]:
gdir="data/genome"
[[ ! -d "$gdir" ]] && mkdir -p "$gdir"

(PZQ-R_field) (PZQ-R_field) 

: 1

In [ ]:
# Download and unzip data
wget -P "$gdir" ftp://ftp.ebi.ac.uk/pub/databases/wormbase/parasite/releases/WBPS14/species/schistosoma_mansoni/PRJEA36577/schistosoma_mansoni.PRJEA36577.WBPS14.genomic.fa.gz
pigz -d "$gdir/schistosoma_mansoni.PRJEA36577.WBPS14.genomic.fa.gz"

# Preparing indices
bwa index "$gdir/schistosoma_mansoni.PRJEA36577.WBPS14.genomic.fa"
samtools faidx "$gdir/schistosoma_mansoni.PRJEA36577.WBPS14.genomic.fa"
gatk CreateSequenceDictionary -R "$gdir/schistosoma_mansoni.PRJEA36577.WBPS14.genomic.fa"

# Download the S. mansoni genome annotation
wget -P "$gdir" ftp://ftp.ebi.ac.uk/pub/databases/wormbase/parasite/releases/WBPS14/species/schistosoma_mansoni/PRJEA36577/schistosoma_mansoni.PRJEA36577.WBPS14.annotations.gff3.gz
pigz -d "$gdir/schistosoma_mansoni.PRJEA36577.WBPS14.annotations.gff3.gz"

In [327]:
# Download and unzip data
wget -P "$gdir" ftp://ftp.ebi.ac.uk/pub/databases/wormbase/parasite/releases/WBPS15/species/schistosoma_haematobium/PRJNA78265/schistosoma_haematobium.PRJNA78265.WBPS15.genomic.fa.gz
pigz -d "$gdir/schistosoma_haematobium.PRJNA78265.WBPS15.genomic.fa.gz"

# Preparing indices
bwa index "$gdir/schistosoma_haematobium.PRJNA78265.WBPS15.genomic.fa"
samtools faidx "$gdir/schistosoma_haematobium.PRJNA78265.WBPS15.genomic.fa"
gatk CreateSequenceDictionary -R "$gdir/schistosoma_haematobium.PRJNA78265.WBPS15.genomic.fa"

# Download the S. mansoni genome annotation
wget -P "$gdir" ftp://ftp.ebi.ac.uk/pub/databases/wormbase/parasite/releases/WBPS15/species/schistosoma_haematobium/PRJNA78265/schistosoma_haematobium.PRJNA78265.WBPS15.annotations.gff3.gz
pigz -d "$gdir/schistosoma_haematobium.PRJNA78265.WBPS15.annotations.gff3.gz"

(PZQ-R_field) --2021-02-12 16:36:32--  ftp://ftp.ebi.ac.uk/pub/databases/wormbase/parasite/releases/WBPS15/species/schistosoma_haematobium/PRJNA78265/schistosoma_haematobium.PRJNA78265.WBPS15.genomic.fa.gz
           => ‘data/genome/schistosoma_haematobium.PRJNA78265.WBPS15.genomic.fa.gz’
Resolving ftp.ebi.ac.uk... 193.62.193.138
Connecting to ftp.ebi.ac.uk|193.62.193.138|:21... connected.
Logging in as anonymous ... Logged in!
==> SYST ... done.    ==> PWD ... done.
==> TYPE I ... done.  ==> CWD (1) /pub/databases/wormbase/parasite/releases/WBPS15/species/schistosoma_haematobium/PRJNA78265 ... done.
==> SIZE schistosoma_haematobium.PRJNA78265.WBPS15.genomic.fa.gz ... 113732870
==> PASV ... done.    ==> RETR schistosoma_haematobium.PRJNA78265.WBPS15.genomic.fa.gz ... done.
Length: 113732870 (108M) (unauthoritative)

schistosoma_haemato 100%[===================>] 108.46M  2.01MB/s    in 58s     

2021-02-12 16:37:36 (1.89 MB/s) - ‘data/genome/schistosoma_haematobium.PRJNA78265.WBPS15.ge

: 1

### Known variants

List of variants from the SmLE x SmHR crosses that segregated in a Mendelian fashion in F1 ([Valentim *et al.* 2013](https://doi.org/10.1126/science.1243106)). This list was generated from alignments using the v5 genome. This has been lifted over to the latest genome using the [flo pipeline](https://github.com/wurmlab/flo/tree/727f10b2b1c57a0514835d302d7f6345d3a34ffb).

In [ ]:
# Index file
gatk IndexFeatureFile -I "$gdir/sm_dbSNP_v7.vcf"

In [ ]:
# For Freebayes
bgzip -c "$gdir/sm_dbSNP_v7.vcf" > "$gdir/sm_dbSNP_v7.vcf.gz"
tabix "$gdir/sm_dbSNP_v7.vcf.gz"

### Population files

These populations files help to defined populations during variant calling with FreeBayes.

In [ ]:
find "$ldir_sm" -type d -name "*SAM*"   | sed "s|.*/||g" | sed "s|$|\tUganda|g" > data/Sm_pop
find "$ldir_sm" -type d -name "*Sm.TZ*" | sed "s|.*/||g" | sed "s|$|\tTanzania|g" >> data/Sm_pop
find "$ldir_sm" -type d -name "*Sm.SN*" | sed "s|.*/||g" | sed "s|$|\tSenegal|g" >> data/Sm_pop
find "$ldir_sm" -type d -name "*Sm.NE*" | sed "s|.*/||g" | sed "s|$|\tNiger|g" >> data/Sm_pop
find "$ldir_sm" -type d -name "*Sm.BR*" | sed "s|.*/||g" | sed "s|$|\tBrazil|g" >> data/Sm_pop
find "$ldir_sm" -type d -name "*Sm.OM*" | sed "s|.*/||g" | sed "s|$|\tOman|g" >> data/Sm_pop
#find "$ldir_sm" -type d -name *Sr* | sed "s|.*/||g" | sed "s|$|\tSro|g" >> data/Sm_pop

In [11]:
find "$ldir_sh" -type d -name "*Sh.TZ*" | sed "s|.*/||g" | sed "s|$|\tTanzania|g" > data/Sh_pop
find "$ldir_sh" -type d -name "*Sh.NE*" | sed "s|.*/||g" | sed "s|$|\tNiger|g" >> data/Sh_pop

(PZQ-R_field) (PZQ-R_field) 

: 1

### Gene GFF files

GFF file gene specific is required for some downstream analysis. This will be generated for *S. mansoni* and *S. haematobium* $TRP_{PZQ}$.

In [ ]:
# Gene code
mygene=Smp_246790
grep "$mygene" "$gdir/schistosoma_mansoni.PRJEA36577.WBPS14.annotations.gff3" > "$gdir/$mygene.gff"

# Generate isoform specific gff
myiso=$(grep -o "$mygene\.[0-9]*" "$gdir/$mygene.gff" | sort | uniq)

for i in $myiso
do
    awk '$3 == "gene"' "$gdir/$mygene.gff" > "$gdir/$i.gff"
    grep -w "$i" "$gdir/$mygene.gff" | grep -v "exon" >> "$gdir/$i.gff"
done

In [14]:
# Gene code
mysh_gene=MS3_0012599
grep "$mysh_gene" "$gdir/schistosoma_haematobium.PRJNA78265.WBPS15.annotations.gff3" > "$gdir/$mysh_gene.gff"

# Generate isoform specific gff
myiso=$(grep -o "$mysh_gene\.[0-9]*" "$gdir/$mysh_gene.gff" | sort | uniq)

for i in $myiso
do
    awk '$3 == "gene"' "$gdir/$mysh_gene.gff" > "$gdir/$i.gff"
    grep -w "$i" "$gdir/$mysh_gene.gff" | grep -v "exon" >> "$gdir/$i.gff"
done

(PZQ-R_field) (PZQ-R_field) (PZQ-R_field) (PZQ-R_field) (PZQ-R_field) (PZQ-R_field) (PZQ-R_field) (PZQ-R_field) (PZQ-R_field) 

: 1

## Sequencing data processing

### Snakemake pipeline

The snakemake pipeline performs the following steps:
* alignment of each library against the reference genome
* marking duplicates
* base quality recalibration (for *S mansoni* only)
* generating alignment statistics
* outputing $TRP_{PZQ}$ gene read depth for each library (*S. mansoni*: Smp_246790; *S. haematobium*: MS3_0012599)
* outputing Z chromosome read depth for each library (for *S mansoni* only)
* determining sex of each individual (for *S mansoni* only)

In [4]:
# Status directory
statdir=status
[[ ! -d "$statdir" ]] && mkdir "$statdir"

(PZQ-R_field) (PZQ-R_field) (PZQ-R_field) 

: 1

#### *S. mansoni*

In [ ]:
snakemake --snakefile Sm.smk --cluster "qsub -V -cwd -o $statdir -j y -r y -pe smp 10 -S /bin/bash" --jobs 1000 -w 120

#### *S. haematobium*

In [ ]:
snakemake --snakefile Sh.smk --cluster "qsub -V -cwd -o $statdir -j y -r y -pe smp 10 -S /bin/bash" --jobs 1000 -w 120

### Variant calling

We call the variants within the gene region across all samples directly from the BAM files using FreeBayes. This method is preferred because it can perform population defined calling.

In [5]:
# Output directory
cdir="data/variant_calling"
[[ ! -d "$cdir" ]] && mkdir -p "$cdir"

(PZQ-R_field) (PZQ-R_field) (PZQ-R_field) 

: 1

#### *S. mansoni*

##### Permissive calling

In [ ]:
# Variables
mygenome="$gdir/schistosoma_mansoni.PRJEA36577.WBPS14.genomic.fa"
#mypos=SM_V7_3:600000-1000000
mypos=SM_V7_3:630000-800000

myfiles=$(find "$ldir_sm" -name *SAM*.bam -o -name *Sm*.bam | sort | tr "\n" " ") ## !! TO CHANGE SAMPLES

myfilename="$cdir/PZQ-R_field"


# Command used for calling variants
freebayes -f "$mygenome" \
   -r "$mypos" \
   -b $myfiles \
   -=          \
   --population data/Sm_pop \
   -q 20       \
   -m 30       \
   -@ "$gdir/sm_dbSNP_v7.vcf.gz" > "${myfilename}.vcf"
   
#     -F 0.4      \

pigz "${myfilename}".vcf

##### Stringent calling

In [17]:
# Variables
mygenome="$gdir/schistosoma_mansoni.PRJEA36577.WBPS14.genomic.fa"
#mypos=SM_V7_3:600000-1000000
mypos=SM_V7_3:630000-800000

myfiles=$(find "$ldir" -name *SAM*.bam -o -name *Sm*.bam | sort | tr "\n" " ") ## !! TO CHANGE SAMPLES

myfilename="$cdir/PZQ-R_field_restrictive"


# Command used for calling variants
freebayes -f "$mygenome" \
   -r "$mypos" \
   -b $myfiles \
   -=          \
   --population data/Sm_pop \
   -q 20       \
   -m 30       \
   -F 0.3      \
   --min-coverage 4 \
   -@ "$gdir/sm_dbSNP_v7.vcf.gz" > "${myfilename}.vcf"
  


pigz "${myfilename}".vcf

(PZQ-R_field) (PZQ-R_field) (PZQ-R_field) (PZQ-R_field) (PZQ-R_field) (PZQ-R_field) (PZQ-R_field) (PZQ-R_field) (PZQ-R_field) (PZQ-R_field) (PZQ-R_field) 
(PZQ-R_field) 

In [9]:
# Variables
mygenome="$gdir/schistosoma_mansoni.PRJEA36577.WBPS14.genomic.fa"
#mypos=SM_V7_3:600000-1000000
mypos=SM_V7_3:630000-800000

myfiles=$(find "$ldir" -name *SAM*.bam -o -name *Sm*.bam | sort | tr "\n" " ") ## !! TO CHANGE SAMPLES

myfilename="$cdir/PZQ-R_field_restrictive2"


# Command used for calling variants
freebayes -f "$mygenome" \
   -r "$mypos" \
   -b $myfiles \
   -=          \
   --population data/Sm_pop \
   -q 20       \
   -m 30       \
   -F 0.3      \
   --min-coverage 10 \
   -@ "$gdir/sm_dbSNP_v7.vcf.gz" > "${myfilename}.vcf"
  


pigz "${myfilename}".vcf

(PZQ-R_field) (PZQ-R_field) (PZQ-R_field) (PZQ-R_field) (PZQ-R_field) (PZQ-R_field) (PZQ-R_field) (PZQ-R_field) (PZQ-R_field) (PZQ-R_field) (PZQ-R_field) WARNING(freebayes): Could not set the region of the variants input file to SM_V7_3:630000..800000
(PZQ-R_field) (PZQ-R_field) (PZQ-R_field) (PZQ-R_field) (PZQ-R_field) 

: 1

In [13]:
echo $gdir

data/genome
(PZQ-R_field) 

: 1

#### *S. haematobium*

In [24]:
# Variables
mygenome="$gdir/schistosoma_haematobium.PRJNA78265.WBPS15.genomic.fa"
mypos=scaffold020018:10279500-10358000

myfiles=$(find "$ldir_sh" -name "*.bam" | sort | tr "\n" " ")

myfilename="$cdir/Sh_PZQ-R_field"


# Command used for calling variants
freebayes -f "$mygenome" \
   -r "$mypos" \
   -b $myfiles \
   -=          \
   --population data/Sh_pop \
   -q 20       \
   -m 30 > "${myfilename}.vcf"

pigz "${myfilename}".vcf

(PZQ-R_field) (PZQ-R_field) (PZQ-R_field) (PZQ-R_field) (PZQ-R_field) (PZQ-R_field) (PZQ-R_field) (PZQ-R_field) (PZQ-R_field) (PZQ-R_field) (PZQ-R_field) (PZQ-R_field) (PZQ-R_field) 

: 1

## Analyzing $SmTRP_{PZQ}$

We will analyze the previously called variants on the gene only. This analysis includes:
* Filtering out genotypes with low read depth,
* Removing invariable sites,
* Normalizing coordinates,
* Use a custom script to summarize variants and genotype from the VCF file.

In [ ]:
myfilename="$cdir/PZQ-R_field"

# Remove genotypes with low read depths
vcftools --gzvcf "${myfilename}.vcf.gz" \
    --minDP 4 \
    --recode  \
    --recode-INFO-all \
    --out "${myfilename}.flt-dp4"

# Rename file
mv "${myfilename}.flt-dp4.recode.vcf" "${myfilename}.flt-dp4.vcf"

# Clean the VCF of invariable ref variants
vcf-subset -a -e "${myfilename}.flt-dp4.vcf" > "${myfilename}.flt-dp4.cleaned.vcf"

# # Normalize VCF
# vt normalize -o "${myfilename}.flt-dp4.cleaned.norm.vcf" \
#     -r "$mygenome" \
#     "${myfilename}.flt-dp4.cleaned.vcf"
## This fails with a core-dump

## Maybe look at bcftools norm (with -f option)   !!! Install bcftools
bcftools norm -o "${myfilename}.flt-dp4.cleaned.norm.vcf" \
    -f "$mygenome" \
    "${myfilename}.flt-dp4.cleaned.vcf"

##Another method of normalization
# vcfleftalign -r "$mygenome" \
#     "${myfilename}.flt-dp4.cleaned.vcf" > "${myfilename}.flt-dp4.cleaned.norm.vcf"

In [3]:
# Result folder
rdir="results/1-reports"
[[ ! -d "$rdir" ]] && mkdir -p "$rdir"

(PZQ-R_field) (PZQ-R_field) (PZQ-R_field) 

: 1

In [ ]:
# Gene and isoforms
mygene=Smp_246790
myiso=$(grep -o "$mygene\.[0-9]*" "$gdir/$mygene.gff" | sort | uniq)

# Generate reports per isoform
for i in $myiso
do
    qsub -V -cwd -o "$statdir" -j y -b y scripts/func-table-mut.sh -v "${myfilename}.flt-dp4.cleaned.norm.vcf" \
        -r "$mygenome" \
        -g "$gdir/$i.gff" \
        -p data/Sm_pop \
        -o "$rdir/$i.flt.norm.tsv"
done

##### Stringent calling

In [ ]:
myfilename="$cdir/PZQ-R_field_restrictive2"

# Remove genotypes with low read depths
vcftools --gzvcf "${myfilename}.vcf.gz" \
    --minDP 4 \
    --recode  \
    --recode-INFO-all \
    --out "${myfilename}.flt-dp4"

# Rename file
mv "${myfilename}.flt-dp4.recode.vcf" "${myfilename}.flt-dp4.vcf"

# Clean the VCF of invariable ref variants
vcf-subset -a -e "${myfilename}.flt-dp4.vcf" > "${myfilename}.flt-dp4.cleaned.vcf"

# # Normalize VCF
# vt normalize -o "${myfilename}.flt-dp4.cleaned.norm.vcf" \
#     -r "$mygenome" \
#     "${myfilename}.flt-dp4.cleaned.vcf"
## This fails with a core-dump

## Maybe look at bcftools norm (with -f option)   !!! Install bcftools
bcftools norm -o "${myfilename}.flt-dp4.cleaned.norm.vcf" \
    -f "$mygenome" \
    "${myfilename}.flt-dp4.cleaned.vcf"

##Another method of normalization
# vcfleftalign -r "$mygenome" \
#     "${myfilename}.flt-dp4.cleaned.vcf" > "${myfilename}.flt-dp4.cleaned.norm.vcf"

In [25]:
myfilename="$cdir/PZQ-R_field_restrictive2"

# Gene and isoforms
mygene=Smp_246790
myiso=$(grep -o "$mygene\.[0-9]*" "$gdir/$mygene.gff" | sort | uniq)


qsub -V -cwd -o "$statdir" -j y -b y scripts/func-table-mut.sh -v "${myfilename}.flt-dp4.cleaned.norm.vcf" \
        -r "$mygenome" \
        -g "$gdir/$mygene.gff" \
        -p data/Sm_pop \
        -o "$rdir/${myfilename##*/}_${mygene}.flt.norm.tsv"

# Generate reports per isoform
for i in $myiso
do
    qsub -V -cwd -o "$statdir" -j y -b y scripts/func-table-mut.sh -v "${myfilename}.flt-dp4.cleaned.norm.vcf" \
        -r "$mygenome" \
        -g "$gdir/$i.gff" \
        -p data/Sm_pop \
        -o "$rdir/${myfilename##*/}_${i}.flt.norm.tsv"
done

(PZQ-R_field) (PZQ-R_field) (PZQ-R_field) (PZQ-R_field) (PZQ-R_field) (PZQ-R_field) (PZQ-R_field) Your job 121955 ("func-table-mut.sh") has been submitted
(PZQ-R_field) (PZQ-R_field) (PZQ-R_field) Your job 121956 ("func-table-mut.sh") has been submitted
Your job 121957 ("func-table-mut.sh") has been submitted
Your job 121958 ("func-table-mut.sh") has been submitted
Your job 121959 ("func-table-mut.sh") has been submitted
Your job 121960 ("func-table-mut.sh") has been submitted
Your job 121961 ("func-table-mut.sh") has been submitted
Your job 121962 ("func-table-mut.sh") has been submitted
(PZQ-R_field) 

: 1

In [9]:
myfilename="$cdir/PZQ-R_field_restrictive2"

# Gene and isoforms
mygene=Smp_246790
myiso=$(grep -o "$mygene\.[0-9]*" "$gdir/$mygene.gff" | sort | uniq)

# Generate reports per isoform
#for i in $myiso
for i in ${mygene}.5
do
    scripts/func-table-mut.sh -v "${myfilename}.flt-dp4.cleaned.norm.vcf" \
        -r "$mygenome" \
        -g "$gdir/$i.gff" \
        -p data/Sm_pop \
        -a pop_cln \
        -o "$rdir/${myfilename##*/}_${i}.flt.norm.tsv"
done

(PZQ-R_field) (PZQ-R_field) (PZQ-R_field) (PZQ-R_field) (PZQ-R_field) (PZQ-R_field) (PZQ-R_field) (PZQ-R_field) Info: Identifying samples related to Tanzania, Senegal, Niger, Brazil, Oman population(s)
Info: Extraction information from GFF
Info: Generating reference sequence
sed: couldn't write 11202 items to stdout: Broken pipe
Progress: [========================================] 100%
(PZQ-R_field) 

: 1

### Global coverage of the gene

We inspect how well the gene and particularly the exons are covered. This is done by concatenating individual coverage of the gene and then plotting total read depth at each position. The concatenation step can take ~20 minutes or more.

In [ ]:
# Result folder
rdir2="results/2-Coverage"
[[ ! -d "$rdir2" ]] && mkdir -p "$rdir2"

# Output file
fl_out="$rdir2/SmTRP-PZQ.cov"

# List files
fl_ls=($(find "$ldir" -type f -name "*PZQ.cov" | sort))

# Create header
echo -e "#CHROM\t$(cut -f1 "${fl_ls[0]}" | tr "\n" "\t" | sed "s/\t$//")" > "$fl_out"
echo -e "POS\t$(cut -f2 "${fl_ls[0]}"| tr "\n" "\t" | sed "s/\t$//")" >> "$fl_out"

for ((i = 0 ; i < ${#fl_ls[@]} ; i++))
do
    echo $i
    echo -e "$(dirname "${fl_ls[$i]}" | sed "s,.*/,,")\t$(cut -f 3 "${fl_ls[$i]}" | tr "\n" "\t" | sed "s/\t$//")"  >> "$fl_out"
done

In [ ]:
# Plotting data

Rscript scripts/SmTRP-PZQ_coverage.R

### Mutations of interest

Some of the mutations are of high interest because located in critical sections of the protein (PZQ binding pocket, channel, TRP box). We list the samples that carry them in order to look at the reads within these samples to assess the likelihood of these mutations. This will also help identify samples that will require further investigation like PCR validation.

In [ ]:
# Result folder
rdir3="results/3-mutations of interest"
[[ ! -d "$rdir3" ]] && mkdir -p "$rdir3"

# mutations="p.T1394I p.Q1432H p.S1448Y p.G1458C p.H1522Q p.I1523N p.P1527T p.P1527Q p.Q1598K p.D1602Y p.D1606Y p.Q1609L p.Q1673K p.M1674I p.D1677Y p.H1680N p.P1683Q p.P1683R p.L1684I p.P1686T p.P1686Q p.P1687Q p.W1692L p.E1696* p.A1700D p.Q1704K"
mutations="p.H1522Q p.I1523N p.P1527T p.P1527Q p.Q1598K p.D1602Y p.D1606Y p.Q1609L p.Q1673K p.M1674I p.D1677Y p.H1680N p.P1686T p.P1686Q p.P1687Q"

# Add stop codon
mutations+=" $(awk '$8 ~ /*/ {print $8}' $rdir/Smp_246790.5.flt.norm.tsv)"

# Isoform of interest
myiso=Smp_246790.5

unset mylist
for m in $mutations
do
echo $m
    mypos=$(awk -v m=$m '$8 == m {print $1}' "$rdir/$myiso.flt.norm.tsv")
    myspl=$(awk -v mypos=$mypos '$2 == mypos' "${myfilename}.flt-dp4.cleaned.norm.vcf"| tr "\t" "\n" | grep -n 0/1 | cut -d ":" -f 1 | tr "\n" "," | sed "s/,$//")
    myspl=$(grep -m 1 "#C" "${myfilename}.flt-dp4.cleaned.norm.vcf" | cut -f $myspl)
    mylist+="$m\t$mypos\t$myspl""\n" 
done

echo -ne "$mylist" > "$rdir3/samples_list.tsv"

In [ ]:
# Output read depth for each sample carrying mutations of interest
Rscript scripts/variant_read_depth.R

Plots of the read depth for each mutation and sample are available in the folder labeled graphs.

### Primer design

Interesting mutations from the calling data must be checked by Sanger sequencing. These mutations are located in specific exons. To sequence the full exons, we extract exon sequences with an additional 200 bp on each end, where primers should be designed.

In [ ]:
fl_out="$rdir3/exons_of_interest"

myexons=(3 4 12 23 25 27 29 34)
myiso="Smp_246790.5"

grep "$myiso" "$gdir/schistosoma_mansoni.PRJEA36577.WBPS14.annotations.gff3" | \
    awk '$3 == "CDS"'  | \
    sed -n "$(sed "s/ /p;/g" <<< ${myexons[@]})p" | \
    awk '{print $1"\t"$4-201"\t"$5+200}' > "$fl_out.bed"

bedtools getfasta -fi "$gdir/schistosoma_mansoni.PRJEA36577.WBPS14.genomic.fa" \
    -bed "$fl_out.bed" \
    -fo "$fl_out.fas"

# Reformat fasta entry
myentries=($(grep -n ">" "$fl_out.fas" | cut -d ":" -f 1))
for ((i = 0 ; i < "${#myexons[@]}" ; i++))
do
    sed -i "${myentries[$i]}s/>/>E${myexons[$i]} /" "$fl_out.fas"
done

### Sanger sequencing

Sanger sequencing traces are analyzed using Consed and PolyPhred: The first align the data while the second perform the calling. PolyPhred output is then processed with a custom script in order to summarize the calling per sample.

In [ ]:
# Data folder
mkdir data/sanger_sequencing

# Download the data

In [ ]:
# Result folder
rdir3b="$rdir3/sanger_sequencing"
[[ ! -d "$rdir3b" ]] && mkdir -p "$rdir3b"

# Phred folders
mkdir -p "$rdir3b/data/"{chromat_dir,edit_dir,phd_dir,poly_dir}

#### Reference sequence

In [ ]:
mygene="Smp_246790"

# Generate reference sequence
mkdir "$rdir3b/0-Ref_seq"
bedtools getfasta -fi "$gdir/schistosoma_mansoni.PRJEA36577.WBPS14.genomic.fa" \
    -bed <(grep "$mygene" "$gdir/schistosoma_mansoni.PRJEA36577.WBPS14.annotations.gff3" | awk '$3 == "gene"' | awk '{print $1"\t"$4-1"\t"$5}') \
    -fo "$rdir3b/Smp_246790.fas"

# Rename fasta entry
sed -i "s/>.*/>$mygene/" "$rdir3/Smp_246790.fas"

In [ ]:
pwd_old=$PWD

# Generate ref sequence for phred
cd "$rdir3b/"
sudophred "$mygene.fas" -r -abi

# Move ref
mv "$mygene.REF" "data/chromat_dir/"
mv "$mygene.REF.phd.1" "data/phd_dir/"

cd "$pwd_old"

#### Prepare data

We create links to the original data but with modified file names in order to informed Polyphred about the paired samples. Samples will be identified on the first 8 characters

In [33]:
# Generate modified sample names for polyphred
j=1
unset spl_code
for i in $(tail -n +2 "$rdir3b/barcode_list.tsv" | cut -f 1 | sort | uniq)
do
    spl_code+="$i\t$(sed "s/^/$(printf "%02d" $j)_/" <<< "$i")\n"
    ((j++))
done

# Generate exon specific folder and link to data
while read line 
do
    # Sample name
    myspl=$(cut -f 1 <<< "$line")
    myspl=$(echo -ne "$spl_code" | grep -w "$myspl" | cut -f2)
    
    # Barcode
    mycode=$(cut -f 3 <<< "$line")
    [[ "$mycode" == NA ]] && continue
    
    # Link files
    ln -s "$PWD/data/sanger_sequencing/$mycode"*.scf "$rdir3b/data/chromat_dir/${myspl}_$mycode.scf"

done < <(tail -n +2 "$rdir3b/barcode_list.tsv")

(PZQ-R_field) (PZQ-R_field) (PZQ-R_field) (PZQ-R_field) (PZQ-R_field) (PZQ-R_field) (PZQ-R_field) 

: 1

Run Phred and Consed

In [36]:
pwd_old=$PWD

# Moving to the working folder
cd "$rdir3b/data/edit_dir"

# Generating phd and poly files
phred ../chromat_dir/*.scf -pd ../phd_dir/ -dd ../poly_dir/

# Preparing the reference sequence
phd2Ace.perl "$mygene.REF.phd.1"

(PZQ-R_field) (PZQ-R_field) (PZQ-R_field) bash: pushd: results/2-mutations of interest/sanger_sequencing/data/edit_dir: No such file or directory
(PZQ-R_field) (PZQ-R_field) (PZQ-R_field)   ../chromat_dir/01_Sm.BR_PdV.1079.1_BXJ481.scf
  ../chromat_dir/01_Sm.BR_PdV.1079.1_BXJ483.scf
  ../chromat_dir/02_Sm.BR_PdV.1371.1_BXJ485.scf
  ../chromat_dir/02_Sm.BR_PdV.1371.1_BXJ487.scf
  ../chromat_dir/03_Sm.BR_PdV.1475_rep_BXJ489.scf
  ../chromat_dir/03_Sm.BR_PdV.1475_rep_BXJ491.scf
  ../chromat_dir/04_Sm.BR_PdV.2196.2_BXJ493.scf
  ../chromat_dir/04_Sm.BR_PdV.2196.2_BXJ495.scf
  ../chromat_dir/05_Sm.BR_PdV.2225.1_BXJ496.scf
  ../chromat_dir/05_Sm.BR_PdV.2225.1_BXJ497.scf
  ../chromat_dir/06_Sm.BR_PdV.2300.1_BXJ498.scf
  ../chromat_dir/06_Sm.BR_PdV.2300.1_BXJ499.scf
  ../chromat_dir/07_Sm.NE_Di186.1_BWK472.scf
  ../chromat_dir/07_Sm.NE_Di186.1_BWK473.scf
  ../chromat_dir/08_Sm.NE_Di68.2_BXJ500.scf
  ../chromat_dir/08_Sm.NE_Di68.2_BXJ501.scf
  ../chromat_dir/09_Sm.OM_She-OD1_BWK488.scf
  ../chro

: 1

In [37]:
# Aligning reads to the reference
find ../chromat_dir/* -name "*.scf" -exec basename {} \; > reads_to_add.fof
consed -ace "$mygene.REF.ace" -addNewReads reads_to_add.fof -newAceFilename "$mygene.REF.ace.1"
cd "$pwd_old"

(PZQ-R_field) (PZQ-R_field) -addNewReads will be run.
no ~/consedrc file so no user resources will be used--that's ok
no consedrc file so no project-specific resources--that's ok
couldn't open readOrder.txt--that's ok
50% done.  1 reads read so far...
Now setting quality values
Number of individual phd files read: 1
Total reads in assembly: 1
Finished setting quality values in 1 seconds 
Consed is detecting that there are 0 forward universal primer reads, 0 reverse universal primer reads, 1 walking reads, 0 pcr end reads, 0 reads of unknown type, and 0 reads did not have their type set in the phd file.  Is this correct?  If not, consed will not pick templates correctly for primers and autofinish will not function correctly.  You probably did not correct customize determineReadTypes.perl to conform to your read naming convention.  I suggest you read README.txt that came with consed and also examine the notes in determineReadTypes.perl.  To help you in figuring out what is wrong, on the 

: 1

In [42]:
cd "/data/infectious/schistosome/06 - PZQ resistance/2021-01-06 PZQ field variants/"
echo $PWD

(PZQ-R_field) /data/infectious/schistosome/06 - PZQ resistance/2021-01-06 PZQ field variants
(PZQ-R_field) 

: 1

Run PolyPhred

In [43]:
score_snp=70 #90
score_indel=70 #80
qual=25 #40

# SNP scoring
name=$mygene.s${score_snp}-q$qual
polyphred -d "$rdir3b/data" -extended_genotype -n $name.snp.nav -o $name.snp.polyphred.out -q $qual -refcomp -s 1 8 -score $score_snp

# Indel scoring
name=$mygene.s${score_indel}-q$qual
polyphred -d "$rdir3b/data" -extended_genotype -i -inav $name.indel.nav -o $name.indel.polyphred.out -q $qual -s 1 8 -iscore $score_indel

(PZQ-R_field) (PZQ-R_field) (PZQ-R_field) (PZQ-R_field) (PZQ-R_field) (PZQ-R_field) 
POLYPHRED  Version 6.18
--------------------------------------------------------------
Writing output to file /data/infectious/schistosome/06 - PZQ resistance/2021-01-06 PZQ field variants/results/2-mutations of interest/sanger_sequencing/data/edit_dir/Smp_246790.s70-q25.snp.polyphred.out

Reading the ACE file /data/infectious/schistosome/06 - PZQ resistance/2021-01-06 PZQ field variants/results/2-mutations of interest/sanger_sequencing/data/edit_dir/Smp_246790.REF.ace.1
Reading contig `Smp_246790.REF-Contig'
Using sequence Smp_246790.REF as reference sequence.
Processing the contig Smp_246790.REF-Contig
   0% done
   1% done (reading sequences)
   2% done (searching for SNPs)
   3% done (searching for SNPs)
   4% done (searching for SNPs)
   5% done (searching for SNPs)
   6% done (searching for SNPs)
   7% done (searching for SNPs)
   8% done (searching for SNPs)
   9% done (searching for SNPs)
  10%

  68% done (searching for SNPs)
  69% done (searching for SNPs)
  70% done (searching for SNPs)
  71% done (searching for SNPs)
  72% done (searching for SNPs)
  73% done (searching for SNPs)
  74% done (searching for SNPs)
  75% done (searching for SNPs)
  76% done (searching for SNPs)
  77% done (searching for SNPs)
  78% done (searching for SNPs)
  79% done (searching for SNPs)
  80% done (searching for SNPs)
  81% done (searching for SNPs)
  82% done (searching for SNPs)
  83% done (searching for SNPs)
  84% done (searching for SNPs)
  85% done (searching for SNPs)
  86% done (searching for SNPs)
  87% done (searching for SNPs)
  88% done (searching for SNPs)
  89% done (searching for SNPs)
  90% done (searching for SNPs)
  91% done (searching for SNPs)
  92% done (searching for SNPs)
  93% done (searching for SNPs)
  94% done (searching for SNPs)
  95% done (searching for SNPs)
  96% done (searching for SNPs)
  97% done (searching for SNPs)
  98% done (searching for SNPs)
  99% do

: 1

In [44]:
# Genotype extractions
scripts/trim_polyphred.sh -i "$rdir3b/data/edit_dir/$mygene.s${score_snp}-q$qual.snp.polyphred.out" \
    -o "$rdir3b/$mygene.SNP.gt.tsv" \
    -p 2 3 \
    -d "_"

scripts/trim_polyphred.sh -i "$rdir3b/data/edit_dir/$mygene.s${score_indel}-q$qual.indel.polyphred.out" \
    -o "$rdir3b/$mygene.INDEL.gt.tsv" \
    -p 2 3 \
    -d "_"


Progress: [========================================] 100%
Progress: [==========                              ] 25%Warning: Several genotype detected for sample A at site 128723. Skipping...
Progress: [====================                    ] 50%Warning: Several genotype detected for sample C at site 128779. Skipping...
Progress: [==============================          ] 75%Warning: Several genotype detected for sample G at site 128806. Skipping...
Progress: [========================================] 100%
(PZQ-R_field) 

: 1

In [47]:
scripts/trim_polyphred.sh -i "$rdir3b/data/edit_dir/$mygene.s${score_indel}-q$qual.indel.polyphred.out" \
    -o "$rdir3b/$mygene.INDEL.gt.tsv" \
    -p 2 3 \
    -d "_"

sed: -e expression #1, char 21: unknown command: `E'
Progress: [==========                              ] 25%Warning: Several genotype detected for sample A at site 128723. Skipping...
Progress: [====================                    ] 50%Warning: Several genotype detected for sample C at site 128779. Skipping...
Progress: [==============================          ] 75%Warning: Several genotype detected for sample G at site 128806. Skipping...
Progress: [========================================] 100%
(PZQ-R_field) 

: 1

## Analyzing $ShTRP_{PZQ}$

We will analyze the previously called variants on the gene only. This analysis includes:
* Filtering out genotypes with low read depth,
* Removing invariable sites,
* Normalizing coordinates,
* Use a custom script to summarize variants and genotype from the VCF file.

In [25]:
myfilename="$cdir/Sh_PZQ-R_field"

# Remove genotypes with low read depths
vcftools --gzvcf "${myfilename}.vcf.gz" \
    --minDP 4 \
    --recode  \
    --recode-INFO-all \
    --out "${myfilename}.flt-dp4"

# Rename file
mv "${myfilename}.flt-dp4.recode.vcf" "${myfilename}.flt-dp4.vcf"

# Clean the VCF of invariable ref variants
vcf-subset -a -e "${myfilename}.flt-dp4.vcf" > "${myfilename}.flt-dp4.cleaned.vcf"

# Maybe look at bcftools norm (with -f option)   !!! Install bcftools
bcftools norm -o "${myfilename}.flt-dp4.cleaned.norm.vcf" \
    -f "$mygenome" \
    "${myfilename}.flt-dp4.cleaned.vcf"

(PZQ-R_field) (PZQ-R_field) (PZQ-R_field) 
VCFtools - 0.1.16
(C) Adam Auton and Anthony Marcketta 2009

Parameters as interpreted:
	--gzvcf data/variant_calling/Sh_PZQ-R_field.vcf.gz
	--recode-INFO-all
	--minDP 4
	--out data/variant_calling/Sh_PZQ-R_field.flt-dp4
	--recode

Using zlib version: 1.2.11
After filtering, kept 96 out of 96 Individuals
Outputting VCF file...
After filtering, kept 1913 out of a possible 1913 Sites
Run Time = 2.00 seconds
(PZQ-R_field) (PZQ-R_field) (PZQ-R_field) (PZQ-R_field) (PZQ-R_field) (PZQ-R_field) (PZQ-R_field) (PZQ-R_field) (PZQ-R_field) Lines total/modified/skipped:	780/57/0
(PZQ-R_field) 

: 1

In [29]:
# Gene and isoforms
mysh_gene=MS3_0012599
myiso=$(grep -o "$mysh_gene\.[0-9]*" "$gdir/$mysh_gene.gff" | sort | uniq)


# Generate reports per isoform
for i in $myiso
do
    qsub -V -cwd -o "$statdir" -j y -b y scripts/func-table-mut.sh -v "${myfilename}.flt-dp4.cleaned.norm.vcf" \
        -r "$mygenome" \
        -g "$gdir/$i.gff" \
        -p data/Sh_pop \
        -o "$rdir/$i.flt.norm.tsv"
done

(PZQ-R_field) (PZQ-R_field) (PZQ-R_field) (PZQ-R_field) (PZQ-R_field) (PZQ-R_field) Your job 33810 ("func-table-mut.sh") has been submitted
(PZQ-R_field) 

: 1

In [30]:
# Result folder
rdir3="results/3-mutations of interest"
[[ ! -d "$rdir3" ]] && mkdir -p "$rdir3"

fl_out="$rdir3/Sh_exons_of_interest"

(PZQ-R_field) (PZQ-R_field) (PZQ-R_field) (PZQ-R_field) (PZQ-R_field) 

: 1

In [32]:
grep "$myiso" "$gdir/schistosoma_haematobium.PRJNA78265.WBPS15.annotations.gff3" | \
    awk '$3 == "CDS"'  > "$fl_out.bed"
#     sed -n "$(sed "s/ /p;/g" <<< ${myexons[@]})p" | \
#     awk '{print $1"\t"$4-201"\t"$5+200}' > "$fl_out.bed"

bedtools getfasta -fi "$gdir/schistosoma_haematobium.PRJNA78265.WBPS15.genomic.fa" \
    -bed "$fl_out.bed" \
    -fo "$fl_out.fas"

(PZQ-R_field) (PZQ-R_field) (PZQ-R_field) (PZQ-R_field) (PZQ-R_field) 

: 1